## 퓨샷 프롬프팅(Few-shot prompting)

Few-shot prompting은 모델에게 **2~5개 정도의 예시(example)**를 함께 제공해서, 모델이 문제 해결 방식이나 답변 형식을 더 잘 이해하고 따라하도록 돕는 기법.

반대로 one-shot prompting은 예시를 딱 한 개만 제공하는 방식으로, 간단한 태스크나 토큰을 아끼고 싶을 때 사용됨.

둘 다 모델이 "패턴을 보고 따라하도록 유도" 한다는 점은 같습니다.

GPT는 이러한 예시를 잘 따라하는 모델임.


In [ ]:
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

In [ ]:
# 복습: ChatOpenAI() 객체는 Runnable로 invoke 바로 실행 가능
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini'
)

question = '대한민국의 수도는 뭐야?'

answer = llm.invoke(question).content
answer

## FewShotPromptTemplate

### 1. 예제 정의: 질문-답변 쌍 구성

모델이 답변 형식으 학습할 수 있도록 하는 예시 질문-답변 쌍을 미리 정의.

퓨샷으로 사용.

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {
        "question": "에베레스트 산과 백두산 중 어느 산이 더 높나요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 에베레스트 산의 높이는 얼마인가요?
중간 답변: 에베레스트 산의 높이는 약 8,848미터입니다.
추가 질문: 백두산의 높이는 얼마인가요?
중간 답변: 백두산의 높이는 약 2,744미터입니다.
최종 답변은: 에베레스트 산
"""
    },
    {
        "question": "서울과 도쿄 중 인구가 더 많은 도시는 어디인가요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 서울의 인구는 몇 명인가요?
중간 답변: 서울의 인구는 약 950만 명입니다.
추가 질문: 도쿄의 인구는 몇 명인가요?
중간 답변: 도쿄의 인구는 약 1,400만 명입니다.
최종 답변은: 도쿄
"""
    },
    {
        "question": "모차르트와 베토벤 중 누가 먼저 태어났나요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 모차르트는 언제 태어났나요?
중간 답변: 모차르트는 1756년에 태어났습니다.
추가 질문: 베토벤은 언제 태어났나요?
중간 답변: 베토벤은 1770년에 태어났습니다.
최종 답변은: 모차르트
"""
    },
    {
        "question": "영국과 프랑스 중 면적이 더 넓은 나라는 어디인가요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 영국의 면적은 얼마인가요?
중간 답변: 영국의 면적은 약 243,000㎢입니다.
추가 질문: 프랑스의 면적은 얼마인가요?
중간 답변: 프랑스의 면적은 약 551,000㎢입니다.
최종 답변은: 프랑스
"""
    },
    {
        "question": "삼국지의 등장인물 중 유비와 조조 중 누가 더 먼저 죽었나요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 유비는 언제 죽었나요?
중간 답변: 유비는 223년에 죽었습니다.
추가 질문: 조조는 언제 죽었나요?
중간 답변: 조조는 220년에 죽었습니다.
최종 답변은: 조조
"""
    },
    {
        "question": "소나타와 교향곡 중 악장이 더 많은 것은 무엇인가요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 일반적인 소나타는 몇 악장으로 구성되나요?
중간 답변: 일반적으로 소나타는 3~4악장으로 구성됩니다.'
추가 질문: 일반적인 교향곡은 몇 악장으로 구성되나요?
중간 답변: 일반적으로 교향곡은 4악장으로 구성됩니다.
최종 답변은: 교향곡
"""
    }
]


### 2. 예시 포맷 정의: 질문과 답변 출력 형태 지정

예시 템플릿이 잘 동작하는지 첫번째 예시를 출력 테스트.

In [ ]:
# 각 예시를 어떻게 출력할지 정의
example_prompt = PromptTemplate.from_template(
    """
    [Question]: {question}
    [Answer]: {answer}
    """
)

print(example_prompt.format(**examples[0]))

### 3. FewShotPromptTemplate 구성: 전체 프롬프트 템플릿 완성

FewShotPromptTemplate으로 프롬프트 생성.

정의된 예시들과 실제 입력 질문을 바탕으로 모델이 이해할 수 있는 완성된 프롬프트를 생성.

- examples: 예시 질문-답변 목록으로, 패턴 학습에 사용됨

- example_prompt: 각 예시를 출력할 때 사용할 템플릿 형식

- suffix: 실제 질문이 들어가는 마지막 프롬프트 영역

- final_prompt: 예시와 suffix가 결합된 최종 프롬프트 문자열

### FewShotPromptTemplate의 suffix는?
- suffix는 예시(examples) 이후에 들어갈 실제 사용자 입력 질문을 포함하는 부분.
- 즉, 프롬프트의 끝부분에 해당하며, 모델이 지금부터 답변해야 할 실제 질문을 제시하는 부분.

![LCEL_exp](https://codetutorbot.blob.core.windows.net/image/suffix.png)

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= 
        """
        [Question]: {question}
        [Answer]: """
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

### 4. LLM 호출

In [ ]:
# 결과 출력
answer = llm.invoke(final_prompt)
print(answer.content)

※ 실습: FewShotPromptTemplate 실습 과제

- 직접 새로운 예제를 추가하고 새로운 질문에 대해 프롬프트를 구성한다.

[1단계. 새로운 예시 추가해보기]

아래 조건을 만족하는 예시 Q&A를 하나 만들어 examples 리스트에 추가하시오.

- 주제: "두 인물 중 누가 더 오래 살았는가?
- 추가 질문과 중간 답변을 반드시 포함할 것
- 최종 답변이 명확히 드러나야 함

![](https://codetutorbot.blob.core.windows.net/image/퓨샷예시.png)

[2단계. 나만의 질문으로 suffix 수정]

아래와 같이 새로운 질문을 하나 작성하고 question 변수에 입력하세요.

![](https://codetutorbot.blob.core.windows.net/image/퓨샷예시1.png)

[마지막. 모델 응답 확인]

llm.invoke(final_prompt)를 호출하여 응답 결과를 확인하고, 아래 질문에 답해보세요.

- LLM의 답변이 적절한가요? (예/아니오)

- 추가 예시를 더 넣으면 어떤 점이 개선될 수 있을까요?

In [ ]:
# YOUR CODE
# 1단계 기존 examples에 새로운 예시 추가하기


# 2단계 나만의 질문으로 suffix 수정하기


# 마지막 모델 응답 확인하기



----

## Example Selector

예제가 많은 경우 프롬프트에 포함할 예제를 선택해야 할 수도 있습니다. 

Example Selector 는 이 작업을 담당하는 클래스.

- 원문: https://python.langchain.com/docs/how_to/example_selectors/

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

📌 1. MaxMarginalRelevanceExampleSelector

+ 기능: 사용자의 질문과 관련성이 높으면서 다양성도 확보되는 예시를 선택합니다.

+ 원리: Max Marginal Relevance (MMR) 알고리즘 사용 
  
  → 질의와 유사한 예시를 선택하면서도 서로 비슷한 예시들이 중복되지 않도록 하는 알고리즘입니다.

+ 장점:
    - 중복 예제를 줄이고,

    - 다양한 관점의 예시를 포함시켜,

    - 과적합된 프롬프트를 방지합니다.

+ 예시:
"구글 창립 연도와 관련된 질문"이 들어오면,
관련성 높은 예시 중 비슷하지 않은 여러 가지를 선택하여 LLM에 전달합니다.

📌 2. SemanticSimilarityExampleSelector

+ 기능: 사용자 입력과 의미상 유사한 예시를 선택합니다.

+ 원리: 모든 예제와 입력 질문을 임베딩(벡터화)한 후, 코사인 유사도 기준으로 가장 가까운 예제를 선택합니다.

+  장점:
    - 빠르고 단순하며,
    - 주어진 예시 중에서 가장 관련성 높은 예시를 잘 찾아냅니다.

✅ from langchain_openai import OpenAIEmbeddings

+ 기능: 텍스트 데이터를 **임베딩 벡터(숫자 벡터)**로 변환하는 OpenAI 모델 래퍼입니다.

+ 예를 들어, text-embedding-ada-002 같은 모델을 사용해서 문장을 임베딩합니다.

+ 예제 선택기와 함께 사용되어, 입력 질문과 예시의 유사도 계산에 활용됩니다.

✅ from langchain_chroma import Chroma

+ 기능: LangChain에서 Chroma 벡터 데이터베이스를 사용할 수 있게 해주는 래퍼입니다.

+ Chroma는 오픈소스 벡터 스토어로, 텍스트 임베딩을 저장하고 유사도 검색을 빠르게 수행할 수 있습니다.

+ SemanticSimilarityExampleSelector나 MMRExampleSelector와 함께 사용할 수 있습니다.

In [ ]:
# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록
    examples,

    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스
    OpenAIEmbeddings(),

    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorDB 클래스
    Chroma,

    # 이것은 생성할 예시의 수
    k=1,
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 입력과 가장 유사한 예시를 선택
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix= 
        """
        [Question]: {question}
        [Answer]: """,
    input_variables=["question"]
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

이번에는 ExampleSlector를 사용하여 FewShotPromptTemplate을 생성한다.

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix= 
        """
        위의 형식을 따라, 아래 질문에 대해 동일한 형식으로 답해주세요.

        [Question]: {question}
        [Answer]: """,
    input_variables=["question"]
)


# 체인 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.invoke(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
print(answer)

※ 실습: ExampleSelector 실습 과제

- 직접 다음 캡쳐의 코드를 작성해보면서 ExampleSelector 실습을 연습해 봅시다.

![](https://codetutorbot.blob.core.windows.net/image/예제1.png)

![](https://codetutorbot.blob.core.windows.net/image/예제2.png)

In [ ]:
# YOUR CODE